# Sandbox for QRN-RL-GNN

This is an IPython notebook to be used for the following:

1) As a sandbox to test snippets of code during development

2) For demonstration purposes of functionalities

### Imports

In [116]:
from repeaters import RepeaterNetwork
from models import CNN, GNN
from gnn_env import Environment

# Basic usage

Below is a set of examples on some basic functionality of the code to make it more apparent how the code connect to the physical system () at least in a high level way. This is Basic usage 1. On Basic usage 2 the container of the environment for the RL agent is showcased. 

## Basic usage 1: The Quantum repeaters

First lets initialize the network this is done with the `RepeaterNetwork` class. It is parametrized by the number of nodes $n$, the system parameters $\tau, p_e, p_s$ and some other stuff

In [90]:
net = RepeaterNetwork(
                    n=4,
                    p_entangle = 1,
                    p_swap = 1,
                    tau = 1_000,
                    kappa = 1,
                    directed = False,
                    geometry = 'chain',
                    )

The state can be seen anytime with ```self.matrix``` (dictionary) or used gor GNNs with ```self.tensorState()``` (pyG.data). For the first case the state is returned in a dictionary of the form $\texttt{self.matrix} = \big\{(i,j) : [\text{adj}, \text{ent}]\big\}$ where $i,j$ denote the vertices (repeaters) of the edge (links), adj denotes the adjecency (0,1) and ent the links.

In [91]:
net.matrix

{(0, 1): [1, 0.0],
 (1, 2): [1, 0.0],
 (0, 3): [0.0, 0.0],
 (2, 3): [1, 0.0],
 (0, 2): [0.0, 0.0],
 (1, 3): [0.0, 0.0]}

In [92]:
# internal variable controls end-to-end
net.global_state

False

### Operation 1: Entanglement

This is what establishes the local links to be extended to reach end-to-end

In [100]:
def look_for_entanglement(edge):
    print(f'Entanglement at {edge}: {net.getLink(edge=(0,1), linkType = 1)}')

In [94]:
edge = (0,1)
look_for_entanglement(edge=edge)
net.entangle(edge = (0,1))
look_for_entanglement(edge=edge)

Entanglement at (0, 1): 0.0
Entanglement at (0, 1): 1


In [95]:
#now lets reset
net.resetState()
net.matrix

{(0, 1): [1, 0],
 (1, 2): [1, 0],
 (0, 3): [0.0, 0],
 (2, 3): [1, 0],
 (0, 2): [0.0, 0],
 (1, 3): [0.0, 0]}

### Operation 2: Swap

The second operation is performing a swap by "merging" the entanglement values of two edges into one. The rule is that the two edges need to share a repeater. This is what effectivelly extends the links to greater than nearest neighbour distances.

In [98]:
#two entanglements and a swap give an extended link
print(f' Initial entanglement :{net.matrix[(0,2)][1]}')
net.entangle(edge=(0,1))
net.entangle(edge=(1,2))
net.swapAT(1)
print(f' Final entanglement : {net.matrix[(0,2)][1]:.3f}')
net.resetState()

 Initial entanglement :0
 Final entanglement : 1.000


In [99]:
# can also be done with an edge specific swap function
print(f' Initial entanglement :{net.matrix[(0,2)][1]}')
net.entangle(edge=(0,1))
net.entangle(edge=(1,2))
net.swap(edge1=(0,1), edge2=(1,2))
print(f' Final entanglement :{net.matrix[(0,2)][1]}')
net.resetState()

 Initial entanglement :0
 Final entanglement :0.0


### Operation 3: Ageing

Yes even quantum networks age

In [103]:
#links age with half life net.tau
[net.entangle(edge=edge) for edge in net.matrix.keys()]
net.tick(T = net.tau)
net.matrix

{(0, 1): [1, 0.3660446348040154],
 (1, 2): [1, 0.3664108625221595],
 (0, 3): [0.0, 0.0],
 (2, 3): [1, 0.36714441755772104],
 (0, 2): [0.0, 0.0],
 (1, 3): [0.0, 0.0]}

## Basic Usage 2: The Environment

This is a container for `net` where a model based RL algorithm acts on the system using a neural network as its environment model.

### Setup:

Here lets create some configuration files. The first one is for the quantum repeater network ,Here we specify the type of network that we want to use. The number of nodes for training and testing can be different. Afterwards we do the same for the agent. Keep in mind here that the hyper parameters have not been optimized with some HPO procedure and therefore results of training may vary wildly depending on the HP and the (stochastic) initialization used to run the experiment.

In [ ]:
#Let's now start using config files, first for the network
sys_config = {
    'n_train'        : 4,
    'n_test'         : 4,
    'tau'            : 50_000,
    'p_entangle'     : .85,
    'p_swap'         : .85,
    'kappa'          : 1, # Global depolarizer, legacy code
    } 

In [ ]:
# Then for the environment
agent_config = {
    'train_agent'    : True,
    'train_steps'    : 50_000,
    'learning_rate'  : 0.005,
    'weight_decay'   : 1e-4,
    'temperature'    : .8,
    'gamma'          : 0.9,
    'epsilon'        : 0.1,
    'plot_metrics'   : True,
    'plot_loss'      : True,
    'print_model'    : True,
    'evaluate_agent' : True,
    'test_steps'     : 1_000,
    'render_eval'    : True,   
    }         

In [104]:
# and lastly for the model used
model_config = {
    'input_features' : 1, # always
    'embedding_dim'  : 8,
    'num_layers'     : 3,
    'num_heads'      : 2,
    'hidden_dim'     : 64, 
    'unembedding_dim': 32, 
    'output_dim'     : 4, # always

}

In [ ]:
# Lets now create instances of the model and Environment
model = GNN(
        node_dim          = model_config['input_features'], 
        embedding_dim     = model_config['embedding_dim'],
        num_layers        = model_config['num_layers'],
        num_heads         = model_config['num_heads'],
        hidden_dim        = model_config['hidden_dim'], 
        unembedding_dim   = model_config['unembedding_dim'], 
        output_dim        = model_config['output_dim'], 
        ) 

exp = Environment(
            model        = model,
            n            = sys_config['n_train'],
            kappa        = sys_config['kappa'],
            tau          = sys_config['tau'],
            p_entangle   = sys_config['p_entangle'], 
            p_swap       = sys_config['p_swap'],
            lr           = agent_config['learning_rate'], 
            weight_decay = agent_config['weight_decay'],
            gamma        = agent_config['gamma'], 
            epsilon      = agent_config['epsilon'],
            temperature  = agent_config['temperature']
        )

### Training:
The agent can be trained by an internal method and will output the metrics on a logs folder

In [ ]:
# if agent_config['train_agent']:
#         exp.trainQ(episodes=agent_config['train_steps'], plot=True)

### Testing:

The agent can be avaluated, either on the same system or on a never seen network with different parameters and output evaluation logs in the same destination as the training function

In [ ]:
# if agent_config['evaluate_agent']:
#         for kind in ['trained', 'random', 'alternating']:
#             exp.test(n_test=sys_config['n_test'], 
#                         max_steps=agent_config['test_steps'], 
#                         kind=kind, 
#                         plot=agent_config['render_eval'])